In [1]:
!jupyter nbconvert --to python '/content/Copy_of_DL_Model_2_VAE.ipynb' --output vae_model.py
!jupyter nbconvert --to python '/content/question_1.ipynb' --output ae_model.py
!jupyter nbconvert --to python '/content/maskedAE.ipynb' --output mae_model.py

[NbConvertApp] Converting notebook /content/Copy_of_DL_Model_2_VAE.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][15]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stderr',
 'output_type': 'stream',
 'text': '100%|██████████| 170M/170M [00:04<00:00, 40.1MB/s]\n'}
[NbConvertApp] Writing 11101 bytes to /content/vae_model.py
[NbConvertApp] Converting notebook /content/question_1.ipynb to python
[NbConvertApp] Writing 10638 bytes to /content/ae_model.py
[NbConvertApp] Converting notebook /content/maskedAE.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][13]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stderr',
 'output_type': 'stream',
 'text': '100%|██████████| 

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import os
import torchvision.datasets as datasets
from vae_model import VAE
from ae_model import ConvAutoencoder
from mae_model import MCAE

In [3]:
transform = transforms.Compose([transforms.ToTensor()])
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)
images, _ = next(iter(test_loader))
images = images.to("cpu")

100%|██████████| 170M/170M [00:10<00:00, 15.7MB/s]


In [4]:
latent_dim=128

In [5]:
vae_model = VAE(latent_dim)
ae_model = ConvAutoencoder()
mae_model = MCAE()

In [6]:
vae_model.eval()

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (fc_mu): Linear(in_features=2048, out_features=128, bias=True)
  (fc_logvar): Linear(in_features=2048, out_features=128, bias=True)
  (decoder_fc): Linear(in_features=128, out_features=2048, bias=True)
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): Sigmoid()
  )
)

In [7]:
ae_model.eval()

ConvAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (5): Sigmoid()
  )
)

In [8]:
mae_model.eval()

MCAE(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Sigmoid()
  )
)

In [11]:
# Unnormalize function
def unnormalize(tensor):
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).to(tensor.device).view(1, 3, 1, 1)
    std = torch.tensor([0.2023, 0.1994, 0.2010]).to(tensor.device).view(1, 3, 1, 1)
    return tensor * std + mean

In [12]:
with torch.no_grad():
    recon_vae = vae_model(images)[0]  # Assuming VAE returns (reconstruction, mu, logvar)
    recon_vae = unnormalize(recon_vae)
    recon_ae = ae_model(images)
    recon_mae = mae_model(images)

In [13]:
def compute_metrics(original, reconstruction):
    # Convert sparse tensors to dense
    if original.is_sparse:
        original = original.to_dense()
    if reconstruction.is_sparse:
        reconstruction = reconstruction.to_dense()

    # Remove batch dimension if present
    if original.dim() == 4:
        original = original[0]
    if reconstruction.dim() == 4:
        reconstruction = reconstruction[0]

    # Convert from (C, H, W) to (H, W, C)
    original_np = original.permute(1, 2, 0).cpu().numpy()
    reconstructed_np = reconstruction.permute(1, 2, 0).cpu().numpy()

    # Handle grayscale images (C=1): squeeze last dim
    if original_np.shape[-1] == 1:
        original_np = original_np.squeeze(-1)
        reconstructed_np = reconstructed_np.squeeze(-1)
        channel_axis = None
    else:
        channel_axis = -1

    # Determine suitable win_size (odd <= min(image dim))
    min_dim = min(original_np.shape[0], original_np.shape[1])
    win_size = min(7, min_dim if min_dim % 2 == 1 else min_dim - 1)

    # Compute metrics
    mse = np.mean((original_np - reconstructed_np) ** 2)
    ssim_val = ssim(original_np, reconstructed_np, win_size=win_size, channel_axis=channel_axis, data_range=original_np.max() - original_np.min())
    psnr_val = psnr(original_np, reconstructed_np, data_range=original_np.max() - original_np.min())

    return mse, ssim_val, psnr_val

In [14]:
idx = 0
orig_img = images[idx]
vae_img = recon_vae[idx]
ae_img = recon_ae[idx]
mae_img = recon_mae[idx]

In [15]:
mse_v, ssim_v, psnr_v = compute_metrics(orig_img, vae_img)
mse_a, ssim_a, psnr_a = compute_metrics(orig_img, ae_img)
mse_m, ssim_m, psnr_m = compute_metrics(orig_img, mae_img)

In [16]:
print("\n--- Quantitative Metrics ---")
print("VAE Model:")
print(f"  MSE:  {mse_v:.4f}")
print(f"  SSIM: {ssim_v:.4f}")
print(f"  PSNR: {psnr_v:.4f}")

print("\nAE Model:")
print(f"  MSE:  {mse_a:.4f}")
print(f"  SSIM: {ssim_a:.4f}")
print(f"  PSNR: {psnr_a:.4f}")

print("\nMAE Model:")
print(f"  MSE:  {mse_m:.4f}")
print(f"  SSIM: {ssim_m:.4f}")
print(f"  PSNR: {psnr_m:.4f}")


--- Quantitative Metrics ---
VAE Model:
  MSE:  0.0944
  SSIM: 0.0778
  PSNR: 7.6467

AE Model:
  MSE:  0.0584
  SSIM: 0.0830
  PSNR: 9.7360

MAE Model:
  MSE:  0.0599
  SSIM: 0.0830
  PSNR: 9.6222
